In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

#from ucimlrepo import fetch_ucirepo, list_available_datasets

from csv import reader
from math import sqrt
plt.rcParams['figure.figsize'] = [16, 10]

import random
from random import seed
from random import randrange
import requests
import io
    
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fac" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fac = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+")

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fou" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fou = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+")

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-zer" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
zer = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+")

In [4]:
print(fac.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Columns: 216 entries, 98 to 13.6
dtypes: int64(216)
memory usage: 3.3 MB
None
